Things that need to be done:
* create IAM role for boto, 

You now want to give boto account the following permisions:
* full ec2 creation access - or just enough to spin it up and run ssm commands on it
* create a policy that allows the boto account to assign a ssm role to ec2 instance

Create a role to assign ec2 instances:
* this role should give the ec2 instance permision to access the ssm API



NOTE: 
* You need your RTC time to be synched to the amazon UTC time
* Your local time can be set to your local time (GTM zone)

In [1]:
import boto3

In [2]:
s3 = boto3.resource('s3')

In [3]:
## if on prem- you need to manualy set the machines time to gmt+0
## -2 hours
# thanks ITO
# 
# date -s '07 June 2018 13:24:00'

In [4]:
for bucket in s3.buckets.all():
    print(bucket.name)

jeroens-bucket
lauren-backups


In [5]:
ec2 = boto3.client('ec2')

In [6]:

#IAM_profile = {
#        'Arn': 'arn:aws:iam::680041685499:instance-profile/SSM-access'
#    }

In [7]:
IAM_profile = {
        'Arn': 'arn:aws:iam::937520633893:instance-profile/EC2=S3+SSM'
    }

In [8]:
tags = [
                {
                    'ResourceType':'instance',
                    'Tags': [
                        {
                            'Key': 'use',
                            'Value': 'boto3'
                        },
                    ]
                }]

* if you want to run an isntance with an IAM role - the boto credentials need to have some admin rights
https://stackoverflow.com/questions/27266400/launching-ec-instance-a-client-error-unauthorizedoperation-occurred-when-ca

For a more in depth explination:
* https://aws.amazon.com/blogs/security/granting-permission-to-launch-ec2-instances-with-iam-roles-passrole-permission/

In [13]:
# defualt ami - AWS LINUX
ami='ami-ca0135b3'

#key='testing-jeroen'
#subnet=subnet-36539650

#ami='ami-59696920'
key='jeroen-R-studio'
subnet='subnet-17a89c4f'
security_group=['sg-1c540361']


In [14]:
instance = ec2.run_instances(
        ImageId=ami,
        MinCount=1,
        MaxCount=1,
        KeyName=key,
        InstanceType='t1.micro',
        SubnetId=subnet,
        TagSpecifications=tags,
        IamInstanceProfile=IAM_profile,
        SecurityGroupIds=security_group
        )

In [13]:
isntance

{'Groups': [],
 'Instances': [{'AmiLaunchIndex': 0,
   'ImageId': 'ami-59696920',
   'InstanceId': 'i-023b3ecd69692fabc',
   'InstanceType': 't1.micro',
   'KeyName': 'jeroen-R-studio',
   'LaunchTime': datetime.datetime(2018, 6, 8, 5, 12, 58, tzinfo=tzutc()),
   'Monitoring': {'State': 'disabled'},
   'Placement': {'AvailabilityZone': 'eu-west-1a',
    'GroupName': '',
    'Tenancy': 'default'},
   'PrivateDnsName': 'ip-172-31-47-66.eu-west-1.compute.internal',
   'PrivateIpAddress': '172.31.47.66',
   'ProductCodes': [],
   'PublicDnsName': '',
   'State': {'Code': 0, 'Name': 'pending'},
   'StateTransitionReason': '',
   'SubnetId': 'subnet-17a89c4f',
   'VpcId': 'vpc-ae7498c9',
   'Architecture': 'x86_64',
   'BlockDeviceMappings': [],
   'ClientToken': '',
   'EbsOptimized': False,
   'Hypervisor': 'xen',
   'IamInstanceProfile': {'Arn': 'arn:aws:iam::937520633893:instance-profile/EC2=S3+SSM',
    'Id': 'AIPAJUSMUGO34SJXZSCA6'},
   'NetworkInterfaces': [{'Attachment': {'AttachTime

In [18]:
def execute_commands_on_linux_instances(client, commands, instance_ids):
    """Runs commands on remote linux instances
    :param client: a boto/boto3 ssm client
    :param commands: a list of strings, each one a command to execute on the instances
    :param instance_ids: a list of instance_id strings, of the instances on which to execute the command
    :return: the response from the send_command function (check the boto3 docs for ssm client.send_command() )
    """

    resp = client.send_command(
        DocumentName="AWS-RunShellScript", # One of AWS' preconfigured documents
        Parameters={'commands': commands},
        InstanceIds=instance_ids,
        OutputS3BucketName='jeroens-bucket',
        OutputS3KeyPrefix='boto-logs'
    )
    return resp

In [23]:
# Example use:
ssm_client = boto3.client('ssm') # Need your credentials here
# command = 'python get-tweets-data-ranges.py --querysearch "etherium" --days "2017-06-02;2017-06-03"'
#command = "echo 'User:'"
#command1 = 'su ec2-user -c "whoami"'
#command2 = 'su ec2-user -c "ls -l ~/GetOldTweets-python"'
#command3 = 'su ec2-user -l -c "conda list env"'
#command4 = "conda"
#commandTimeoutSeconds5 = 'su ec2-user -c "which python"'
#command3 = 'echo "python ~/GetOldTweets-python/crypto_v2.py --querysearch \'etherium\' --days \'2017-06-02;2017-06-03\'"'

command = 'su ec2-user -l -c "python ~/GetOldTweets-python/crypto_v2.py --querysearch \'etherium\' --days \'2017-06-04;2017-06-05\'"'

#command2 = 'su ec2-user -l -c "python ~/GetOldTweets-python/crypto_v2.py --querysearch ''\'' 
#"etherium'\'' --days ""2017-06-02;2017-06-03"""'  


commands = [command]
instance_ids = ['i-01b2c6f83b28b5f39']
execute_commands_on_linux_instances(ssm_client, commands, instance_ids)

{'Command': {'CommandId': '30dcf321-2b00-4e2d-9561-4de277f5ab92',
  'DocumentName': 'AWS-RunShellScript',
  'DocumentVersion': '',
  'Comment': '',
  'ExpiresAfter': datetime.datetime(2018, 6, 9, 20, 43, 27, 187000, tzinfo=tzlocal()),
  'Parameters': {'commands': ['su ec2-user -l -c "python ~/GetOldTweets-python/crypto_v2.py --querysearch \'etherium\' --days \'2017-06-04;2017-06-05\'"']},
  'InstanceIds': ['i-01b2c6f83b28b5f39'],
  'Targets': [],
  'RequestedDateTime': datetime.datetime(2018, 6, 9, 18, 43, 27, 187000, tzinfo=tzlocal()),
  'Status': 'Pending',
  'StatusDetails': 'Pending',
  'OutputS3BucketName': 'jeroens-bucket',
  'OutputS3KeyPrefix': 'boto-logs',
  'MaxConcurrency': '50',
  'MaxErrors': '0',
  'TargetCount': 1,
  'CompletedCount': 0,
  'ErrorCount': 0,
  'ServiceRole': '',
  'NotificationConfig': {'NotificationArn': '',
   'NotificationEvents': [],
   'NotificationType': ''}},
 'ResponseMetadata': {'RequestId': '965d5c6b-0c00-490f-ba43-ed74cd702906',
  'HTTPStatusCod

In [11]:
print(command3)

su ec2-user -l -c "python ~/GetOldTweets-python/crypto_v2.py --querysearch 'etherium' --days '2017-06-02;2017-06-03'"


In [30]:
'su ec2-user -l -c \'python ~/GetOldTweets-python/crypto_v2.py --querysearch '

"su ec2-user -l -c 'python ~/GetOldTweets-python/crypto_v2.py --querysearch "

In [55]:
x = 'su ec2-user -l -c "python ~/GetOldTweets-python/crypto_v2.py --querysearch \'etherium\' --days \'2017-06-02;2017-06-03\'"'

In [56]:
print(x)

su ec2-user -l -c "python ~/GetOldTweets-python/crypto_v2.py --querysearch 'etherium' --days '2017-06-02;2017-06-03'"


In [27]:
' don\'t '

" don't "

In [17]:
! aws ssm describe-instance-information

{
    "InstanceInformationList": [
        {
            "InstanceId": "i-01b2c6f83b28b5f39",
            "PingStatus": "Online",
            "LastPingDateTime": 1528561812.548,
            "AgentVersion": "2.2.120.0",
            "IsLatestVersion": false,
            "PlatformType": "Linux",
            "PlatformName": "Amazon Linux AMI",
            "PlatformVersion": "2018.03",
            "ResourceType": "EC2Instance",
            "IPAddress": "172.31.5.69",
            "ComputerName": "ip-172-31-5-69.eu-west-1.compute.internal"
        }
    ]
}


# Commands

In [ ]:
# set up proccess
commands = [
'yum update -y',
'yum install git -y',
'wget https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh',
'chmod 777 Miniconda2-latest-Linux-x86_64.sh',
'bash ./Miniconda2-latest-Linux-x86_64.sh -b',
'echo "export PATH=/root/miniconda2/bin:$PATH" >> /root/.bashrc ',
'conda install boto -y',
'sudo yum groupinstall "Development Tools" -y',
'yum install python-lxml',
'yum install libxml2-devel libxslt-devel python-devel',
'git clone https://github.com/Jefferson-Henrique/GetOldTweets-python.git',
'pip install -r requirements.txt',
'mkdir /GetOldTweets-python/Bitcoin',
'mkdir /GetOldTweets-python/Ethereum',
'aws s3 cp s3://jeroens-bucket/cryto_v2.py GetOldTweets-python/cryto_v2.py'
]

In [16]:
# set up proccess 
command2 = '''
yum update -y
yum install git -y
wget https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh
chmod 777 Miniconda2-latest-Linux-x86_64.sh
bash ./Miniconda2-latest-Linux-x86_64.sh -b
echo "export PATH=/root/miniconda2/bin:$PATH" >> /root/.bashrc 
conda install boto -y
sudo yum groupinstall "Development Tools" -y
yum install python-lxml
yum install libxml2-devel libxslt-devel python-devel
git clone https://github.com/Jefferson-Henrique/GetOldTweets-python.git
pip install -r requirements.txt
mkdir /GetOldTweets-python/Bitcoin
mkdir /GetOldTweets-python/Ethereum
aws s3 cp s3://jeroens-bucket/cryto_v2.py GetOldTweets-python/cryto_v2.py
'''

'\nyum update -y\nyum install git -y\nwget https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh\nchmod 777 Miniconda2-latest-Linux-x86_64.sh\nbash ./Miniconda2-latest-Linux-x86_64.sh -b\necho "export PATH=/root/miniconda2/bin:$PATH" >> /root/.bashrc \nconda install boto -y\nsudo yum groupinstall "Development Tools" -y\nyum install python-lxml\nyum install libxml2-devel libxslt-devel python-devel\ngit clone https://github.com/Jefferson-Henrique/GetOldTweets-python.git\npip install -r requirements.txt\nmkdir /GetOldTweets-python/Bitcoin\nmkdir /GetOldTweets-python/Ethereum\naws s3 cp s3://jeroens-bucket/cryto_v2.py GetOldTweets-python/cryto_v2.py\n'